# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from random import sample

import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Daily Prediction_Step by Step/sales_train_validation.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks_Daily Prediction_Step by Step/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Parameters settings

In [7]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100

In [8]:
validation_training_ratio = 0.99

In [9]:
list_params = list()
for p in [0, 1, 2, 3]:
    for d in range(2):
        for q in range(4):
            list_params.append((p, d, q))

In [10]:
list_params

[(0, 0, 0),
 (0, 0, 1),
 (0, 0, 2),
 (0, 0, 3),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 1, 3),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 3),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 1, 3),
 (2, 0, 0),
 (2, 0, 1),
 (2, 0, 2),
 (2, 0, 3),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 1, 3),
 (3, 0, 0),
 (3, 0, 1),
 (3, 0, 2),
 (3, 0, 3),
 (3, 1, 0),
 (3, 1, 1),
 (3, 1, 2),
 (3, 1, 3)]

In [11]:
len(list_params)

32

# This step involves specifying Demand Pattern (Intermittent or Lumpy or Erratic or Smooth)

In [12]:
sales_pattern = sales_lumpy.copy()

In [13]:
sales_pattern_py = sales_pattern.copy()
list_pattern_py = sales_pattern_py.id.unique().tolist()

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
test_data = df_pattern.iloc[n_pred_days*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

# Hyperparameter for ARIMA model

In [14]:
list_sample = sample(list_pattern_py, 50)

In [15]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in list_sample:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_sample.index(product) / len(list_sample) * 100, 2)))
    train_first = train_data_first_part[product].iloc[-num_train_needed:]
    train_second = train_data_second_part[product]
    
    for combi in list_params:
        model = ARIMA(train_first, order = combi)
        model_fit = model.fit()
        forecast_temp = model_fit.forecast(len(train_second))
        df_temp = pd.DataFrame({'Date': train_second.index,
                                'Actual Data': train_second.values,
                                'Forecast': forecast_temp.values,
                                'Combination': [combi for combi_count in range(len(train_second))], 
                                'Product': [product for product_count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_temp, ignore_index=False)
        
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_3_082_CA_2_validation
Progressing: 0.0 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimiz

Currently Running Product: FOODS_3_020_CA_3_validation
Progressing: 2.0 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimiz

Currently Running Product: FOODS_2_277_CA_2_validation
Progressing: 4.0 %


...

Currently Running Product: FOODS_3_629_CA_1_validation
Progressing: 96.0 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_1_066_CA_3_validation
Progressing: 98.0 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


This line of code took 6.247705111966667 minutes


In [16]:
df_result_all_params

,Date,Actual Data,Forecast,Combination,Product
0,2016-04-03,5.0,0.769995,"(0, 0, 0)",FOODS_3_082_CA_2_validation
1,2016-04-04,0.0,0.769995,"(0, 0, 0)",FOODS_3_082_CA_2_validation
2,2016-04-05,2.0,0.769995,"(0, 0, 0)",FOODS_3_082_CA_2_validation
3,2016-04-06,2.0,0.769995,"(0, 0, 0)",FOODS_3_082_CA_2_validation
4,2016-04-07,0.0,0.769995,"(0, 0, 0)",FOODS_3_082_CA_2_validation
...,...,...,...,...,...
3,2016-04-06,1.0,1.128798,"(3, 1, 3)",FOODS_1_066_CA_3_validation
4,2016-04-07,0.0,0.880332,"(3, 1, 3)",FOODS_1_066_CA_3_validation
5,2016-04-08,0.0,1.153181,"(3, 1, 3)",FOODS_1_066_CA_3_validation
6,2016-04-09,0.0,0.908514,"(3, 1, 3)",FOODS_1_066_CA_3_validation


# Summay all metrics

In [18]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [19]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Combination']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.0979629981666657 minutes


In [20]:
df_result_metrics_all_params

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_010_CA_3_validation,"(0, 0, 0)",0.621248,141.9995,179.0711,72.0003,0.709998,1.003195
1,FOODS_1_010_CA_3_validation,"(0, 0, 1)",0.625200,142.9029,179.0457,71.9553,0.714514,1.004962
2,FOODS_1_010_CA_3_validation,"(0, 0, 2)",0.613809,140.2992,179.1185,72.0843,0.701496,1.000197
3,FOODS_1_010_CA_3_validation,"(0, 0, 3)",0.596865,136.4262,179.2254,72.2731,0.682131,0.993744
4,FOODS_1_010_CA_3_validation,"(0, 1, 0)",0.437500,100.0000,NaN,100.0000,0.500000,1.118034
...,...,...,...,...,...,...,...,...
1595,FOODS_3_803_CA_1_validation,"(3, 0, 3)",1.060261,86.5519,66.7633,74.6116,1.514659,1.990453
1596,FOODS_3_803_CA_1_validation,"(3, 1, 0)",1.382606,112.8658,78.9321,109.4427,1.975151,2.382484
1597,FOODS_3_803_CA_1_validation,"(3, 1, 1)",1.168181,95.3617,71.6438,86.8670,1.668831,2.098615
1598,FOODS_3_803_CA_1_validation,"(3, 1, 2)",1.321646,107.8894,75.7072,100.5701,1.888065,2.421525


# Check what products has unexpected metrics result

In [21]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
160,FOODS_1_070_CA_2_validation,"(0, 0, 0)",inf,inf,200.0,NaN,1.179995,1.179995
161,FOODS_1_070_CA_2_validation,"(0, 0, 1)",inf,inf,200.0,NaN,1.158221,1.159285
162,FOODS_1_070_CA_2_validation,"(0, 0, 2)",inf,inf,200.0,NaN,1.136715,1.139185
163,FOODS_1_070_CA_2_validation,"(0, 0, 3)",inf,inf,200.0,NaN,1.147609,1.149592
164,FOODS_1_070_CA_2_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1467,FOODS_3_707_CA_3_validation,"(3, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
1468,FOODS_3_707_CA_3_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
1469,FOODS_3_707_CA_3_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000000,0.000000
1470,FOODS_3_707_CA_3_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000000,0.000000


In [22]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
160,FOODS_1_070_CA_2_validation,"(0, 0, 0)",inf,inf,200.0,NaN,1.179995,1.179995
161,FOODS_1_070_CA_2_validation,"(0, 0, 1)",inf,inf,200.0,NaN,1.158221,1.159285
162,FOODS_1_070_CA_2_validation,"(0, 0, 2)",inf,inf,200.0,NaN,1.136715,1.139185
163,FOODS_1_070_CA_2_validation,"(0, 0, 3)",inf,inf,200.0,NaN,1.147609,1.149592
164,FOODS_1_070_CA_2_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1467,FOODS_3_707_CA_3_validation,"(3, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
1468,FOODS_3_707_CA_3_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
1469,FOODS_3_707_CA_3_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000000,0.000000
1470,FOODS_3_707_CA_3_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000000,0.000000


In [23]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
4,FOODS_1_010_CA_3_validation,"(0, 1, 0)",0.437500,100.0,NaN,100.0,0.500,1.118034
68,FOODS_1_044_CA_3_validation,"(0, 1, 0)",0.729167,100.0,NaN,100.0,0.625,1.457738
100,FOODS_1_066_CA_3_validation,"(0, 1, 0)",1.312500,100.0,NaN,100.0,0.375,0.612372
132,FOODS_1_069_CA_1_validation,"(0, 1, 0)",0.875000,100.0,NaN,100.0,0.375,1.060660
164,FOODS_1_070_CA_2_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000,0.000000
...,...,...,...,...,...,...,...,...
1468,FOODS_3_707_CA_3_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000,0.000000
1469,FOODS_3_707_CA_3_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000,0.000000
1470,FOODS_3_707_CA_3_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000,0.000000
1471,FOODS_3_707_CA_3_validation,"(3, 1, 3)",NaN,NaN,NaN,NaN,0.000,0.000000


In [24]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
160,FOODS_1_070_CA_2_validation,"(0, 0, 0)",inf,inf,200.0,NaN,1.179995,1.179995
161,FOODS_1_070_CA_2_validation,"(0, 0, 1)",inf,inf,200.0,NaN,1.158221,1.159285
162,FOODS_1_070_CA_2_validation,"(0, 0, 2)",inf,inf,200.0,NaN,1.136715,1.139185
163,FOODS_1_070_CA_2_validation,"(0, 0, 3)",inf,inf,200.0,NaN,1.147609,1.149592
164,FOODS_1_070_CA_2_validation,"(0, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
1467,FOODS_3_707_CA_3_validation,"(3, 0, 3)",inf,inf,200.0,NaN,0.000005,0.000005
1468,FOODS_3_707_CA_3_validation,"(3, 1, 0)",NaN,NaN,NaN,NaN,0.000000,0.000000
1469,FOODS_3_707_CA_3_validation,"(3, 1, 1)",NaN,NaN,NaN,NaN,0.000000,0.000000
1470,FOODS_3_707_CA_3_validation,"(3, 1, 2)",NaN,NaN,NaN,NaN,0.000000,0.000000


In [25]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [26]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Combination,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [27]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [28]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 26.0 %
Percentage of unexpected values of WMAPE is: 26.0 %
Percentage of unexpected values of SMAPE is: 52.0 %
Percentage of unexpected values of MAPE is: 26.0 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [29]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Combination

In [30]:
df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Combination,,,,,,
"(0, 0, 0)",1.165191,137.702011,123.266159,64.668216,2.019323,2.447641
"(0, 0, 1)",1.161612,137.107346,123.173373,64.580803,2.017036,2.442257
"(0, 0, 2)",1.153886,135.679676,122.769584,64.060670,2.000943,2.425892
"(0, 0, 3)",1.157844,136.143878,121.610576,63.306211,1.992218,2.410745
"(0, 1, 0)",1.073853,100.969412,124.803154,86.608413,3.036458,3.727982
"(0, 1, 1)",1.050092,116.127224,120.661738,66.486200,2.006728,2.457930
"(0, 1, 2)",1.104174,127.635768,119.921881,64.179686,2.022002,2.479583
"(0, 1, 3)",1.093770,126.046562,119.249046,63.858681,2.022308,2.483848
"(1, 0, 0)",1.150913,135.709268,121.346970,63.426505,1.988756,2.417413


In [31]:
for mts in list_metrics:
    print('The optimum Combination based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Combination based on Mean MASE is: (2, 1, 0)
The optimum Combination based on Mean WMAPE is: (3, 1, 0)
The optimum Combination based on Mean SMAPE is: (0, 1, 3)
The optimum Combination based on Mean MAPE is: (1, 0, 2)
The optimum Combination based on Mean MAE is: (2, 0, 0)
The optimum Combination based on Mean RMSE is: (2, 0, 0)


### Get MEDIAN metrics value of each Combination

In [32]:
df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Combination,,,,,,
"(0, 0, 0)",0.868437,94.2859,118.4473,62.99980,1.459997,1.687421
"(0, 0, 1)",0.862647,94.0874,118.4381,66.80670,1.384836,1.696012
"(0, 0, 2)",0.845642,98.2187,118.2457,63.14850,1.312271,1.714542
"(0, 0, 3)",0.840329,98.1205,118.2218,61.38380,1.276471,1.690626
"(0, 1, 0)",0.875000,100.0000,111.4101,99.45085,1.750000,2.091437
"(0, 1, 1)",0.844369,100.0000,112.5681,60.66720,1.250000,1.398002
"(0, 1, 2)",0.861914,97.6069,113.5634,60.24870,1.249639,1.549359
"(0, 1, 3)",0.862487,98.4925,113.8337,60.39070,1.248351,1.619270
"(1, 0, 0)",0.835817,94.5795,118.4355,59.95220,1.259526,1.636593


In [33]:
for mts in list_metrics:
    print('The optimum Combination based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Combination'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Combination based on Median of MASE is: (2, 1, 0)
The optimum Combination based on Median of WMAPE is: (1, 1, 2)
The optimum Combination based on Median of SMAPE is: (3, 1, 3)
The optimum Combination based on Median of MAPE is: (1, 0, 2)
The optimum Combination based on Median of MAE is: (3, 1, 3)
The optimum Combination based on Median of RMSE is: (0, 1, 1)


# Run ARIMA model after decide best paramaters

In [34]:
best_combination = (2, 1, 0)

In [35]:
start = timer()

df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    print('Currently Running Product: %s' % product)
    print('Progressing: {0} %'.format(round(list_pattern_py.index(product) / len(list_pattern_py) * 100, 2)))
    train_product = train_data[product].iloc[num_train_needed*-1:]
    test_product = test_data[product]  
    
    model = ARIMA(train_product, order = best_combination)
    model_fit = model.fit()

    forecast_temp = model_fit.forecast(len(test_product))
    df_result_temp = pd.DataFrame({'Date': test_product.index,
                                   'Actual Data': test_product.values,
                                   'Forecast': forecast_temp.values,
                                   'Product': [product for product_count in range(len(test_product))]})
    
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True) 
    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_1_005_CA_1_validation
Progressing: 0.0 %
Currently Running Product: FOODS_1_013_CA_1_validation
Progressing: 0.08 %
Currently Running Product: FOODS_1_014_CA_1_validation
Progressing: 0.16 %
Currently Running Product: FOODS_1_017_CA_1_validation
Progressing: 0.24 %
Currently Running Product: FOODS_1_019_CA_1_validation
Progressing: 0.31 %
Currently Running Product: FOODS_1_027_CA_1_validation
Progressing: 0.39 %
Currently Running Product: FOODS_1_031_CA_1_validation
Progressing: 0.47 %
Currently Running Product: FOODS_1_032_CA_1_validation
Progressing: 0.55 %
Currently Running Product: FOODS_1_035_CA_1_validation
Progressing: 0.63 %
Currently Running Product: FOODS_1_037_CA_1_validation
Progressing: 0.71 %
Currently Running Product: FOODS_1_039_CA_1_validation
Progressing: 0.78 %
Currently Running Product: FOODS_1_040_CA_1_validation
Progressing: 0.86 %
Currently Running Product: FOODS_1_044_CA_1_validation
Progressing: 0.94 %
Currently Running Product:

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_1_120_CA_1_validation
Progressing: 2.9 %
Currently Running Product: FOODS_1_121_CA_1_validation
Progressing: 2.98 %
Currently Running Product: FOODS_1_122_CA_1_validation
Progressing: 3.06 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_1_124_CA_1_validation
Progressing: 3.13 %
Currently Running Product: FOODS_1_128_CA_1_validation
Progressing: 3.21 %
Currently Running Product: FOODS_1_131_CA_1_validation
Progressing: 3.29 %


...

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_3_792_CA_4_validation
Progressing: 99.14 %
Currently Running Product: FOODS_3_801_CA_4_validation
Progressing: 99.22 %
Currently Running Product: FOODS_3_803_CA_4_validation
Progressing: 99.29 %


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Currently Running Product: FOODS_3_804_CA_4_validation
Progressing: 99.37 %
Currently Running Product: FOODS_3_805_CA_4_validation
Progressing: 99.45 %
Currently Running Product: FOODS_3_808_CA_4_validation
Progressing: 99.53 %
Currently Running Product: FOODS_3_814_CA_4_validation
Progressing: 99.61 %
Currently Running Product: FOODS_3_816_CA_4_validation
Progressing: 99.69 %
Currently Running Product: FOODS_3_819_CA_4_validation
Progressing: 99.76 %
Currently Running Product: FOODS_3_822_CA_4_validation
Progressing: 99.84 %
Currently Running Product: FOODS_3_827_CA_4_validation
Progressing: 99.92 %
This line of code took 1.8397695730499986 minutes


/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [36]:
df_result_best_params

,Date,Actual Data,Forecast,Product
0,2016-04-11,1.0,0.663240,FOODS_1_005_CA_1_validation
1,2016-04-12,0.0,0.552654,FOODS_1_005_CA_1_validation
2,2016-04-13,1.0,0.407593,FOODS_1_005_CA_1_validation
3,2016-04-14,1.0,0.540219,FOODS_1_005_CA_1_validation
4,2016-04-15,0.0,0.500025,FOODS_1_005_CA_1_validation
...,...,...,...,...
17859,2016-04-20,0.0,0.000000,FOODS_3_827_CA_4_validation
17860,2016-04-21,0.0,0.000000,FOODS_3_827_CA_4_validation
17861,2016-04-22,0.0,0.000000,FOODS_3_827_CA_4_validation
17862,2016-04-23,0.0,0.000000,FOODS_3_827_CA_4_validation


In [37]:
#df_result_best_params.to_csv('ARIMA_Lumpy_Point_Forecast.csv')

In [38]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [39]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_001_CA_2_validation,0.728908,87.7328,142.4860,73.7404,1.065326,1.329148
1,FOODS_1_001_CA_3_validation,0.789497,136.0363,183.9238,74.9742,0.971688,1.843460
2,FOODS_1_002_CA_2_validation,0.681440,122.3098,143.5996,50.7820,0.524185,0.527136
3,FOODS_1_003_CA_2_validation,1.000280,76.6026,95.9714,55.6790,2.462227,3.456661
4,FOODS_1_003_CA_3_validation,0.781746,108.2418,178.6133,85.6257,0.541209,0.722067
...,...,...,...,...,...,...,...
1271,FOODS_3_824_CA_1_validation,2.234073,481.1850,193.7995,72.2831,0.343704,0.365761
1272,FOODS_3_824_CA_2_validation,0.464286,100.0000,200.0000,100.0000,0.071429,0.267261
1273,FOODS_3_827_CA_2_validation,NaN,NaN,NaN,NaN,0.000000,0.000000
1274,FOODS_3_827_CA_3_validation,NaN,NaN,NaN,NaN,0.000000,0.000000


In [40]:
df_result_final.RMSE.mean()

1.9705533113255889

In [41]:
df_result_final.MAPE.mean()

79.16806855087358

In [43]:
df_result_final.to_csv('ARIMA_Lumpy_Metrics.csv')